# Measure Sampling Time - pystan


# Select Data, IRT model, and Device


In [ ]:
# 0 -> bone
# 1 (others) -> brain

#DATA = 0 
DATA = 1


In [ ]:
# 0 -> 1PL-IRT
# 1 (others) -> 2PL-IRT 

MODEL = 0
#MODEL = 1

In [ ]:
# 0 -> CPU
# 1 (others) -> GPU

DEVICE = 0


In [ ]:
num_chains = 2


# Prepare

In [ ]:
! cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2200.134
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4400.26
clflush size	: 64
cache_alignment	: 64
addres

In [ ]:
! pip install -q pystan==3.3.0 arviz==0.12.1 nest_asyncio==1.5.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import mstats

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import stan
import arviz as az

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Import Data

In [ ]:
fff = "idata_ppc_for_data%s_model%s.nc" % (DATA, MODEL)

! wget https://filedn.com/lpAczQGgeBjkX6l7SpI5JJy/__ws/stan_irt_nrm_rad/{fff} -O idata_ppc.nc

--2023-03-15 20:00:44--  https://filedn.com/lpAczQGgeBjkX6l7SpI5JJy/__ws/stan_irt_nrm_rad/idata_ppc_for_data1_model0.nc
Resolving filedn.com (filedn.com)... 74.120.9.25
Connecting to filedn.com (filedn.com)|74.120.9.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38241044 (36M) [application/x-netcdf]
Saving to: ‘idata_ppc.nc’

idata_ppc.nc        100%[===================>]  36.47M  5.61MB/s    in 9.0s    

2023-03-15 20:00:53 (4.04 MB/s) - ‘idata_ppc.nc’ saved [38241044/38241044]



In [ ]:
idata_ppc = az.from_netcdf('idata_ppc.nc')

In [ ]:
y_ppc = idata_ppc.posterior_predictive['obs'].to_numpy()
y_ppc.shape

(6, 3000, 42, 14)

## Define Model

In [ ]:
stan_code_1pl = """
data {

  int<lower=1> n_person;      //number of radiologists
  int<lower=1> n_observation; //number of observations (== number of patients)

  array[n_observation, n_person] int response;

  real<lower=0> S1;
  real<lower=0> S2;
}
 
parameters {
  array[n_person] real theta;       //ability
  array[n_observation] real beta;   //difficulty
}
 
model {

  //prior
  theta ~ normal(0,S1);
  beta ~ normal(0,S2);

  //main
  for (o in 1:n_observation){
    for(p in 1:n_person){
      response[o,p] ~ bernoulli_logit(theta[p] - beta[o]);
    }
  }
}

"""

In [ ]:
stan_code_2pl = """
data {

  int<lower=1> n_person;      //number of radiologists
  int<lower=1> n_observation; //number of observations (== number of patients)

  array[n_observation, n_person] int response;

  real<lower=0> S1;
  real<lower=0> S2;
}
 
parameters {
  array[n_person] real theta;                      //ability
  array[n_observation] real beta;                  //difficulty
  array[n_observation] real log_discriminatives;   //discriminative
}
 
model {

  //prior
  theta ~ normal(0,S1);
  beta ~ normal(0,S2);
  log_discriminatives ~ normal(0.5,1);

  //main
  for (o in 1:n_observation){
    for(p in 1:n_person){
      response[o,p] ~ bernoulli_logit( exp(log_discriminatives[o]) * (theta[p] - beta[o]) );
    }
  }
}

"""

In [ ]:
stan_code = stan_code_1pl if MODEL == 0 else stan_code_2pl

## Inference

## Measure Inference Time

In [ ]:
import datetime as dt
import time

if MODEL == 0:
    factors = [1, 1, 2, 5, 10, 20, 50] 
else:
    factors = [1, 1, 2, 5, 10, 20]

num_doctors = y_ppc.shape[-1]

In [ ]:
lines = []

for k, factor in enumerate(factors):
    y_simulated = y_ppc[0, :factor, :, :].reshape(-1, num_doctors)
    print("simulated data shape:", y_simulated.shape)
    num_cases = y_simulated.shape[0]
    start_time = dt.datetime.now()

    stan_data = dict(
        n_person=num_doctors, 
        n_observation=num_cases, 
        response=y_simulated,
        S1=2,
        S2=2
    )
    posterior = stan.build(stan_code, data=stan_data, random_seed=123)
    fit = posterior.sample(num_chains=num_chains, num_samples=3000, num_warmup=500)

    end_time = dt.datetime.now()
    elapsed_time = (end_time - start_time).total_seconds()
    lines.append(f'{num_cases},{start_time},{end_time},{elapsed_time}')

    time.sleep(1) # sleep 1 sec

simulated data shape: (42, 14)
Building...



Building: 45.2s, done.Messages from stanc:
Sampling:   0%
Sampling:   0% (1/7000)
Sampling:   0% (2/7000)
Sampling:   1% (101/7000)
Sampling:   3% (200/7000)
Sampling:   4% (300/7000)
Sampling:   6% (400/7000)
Sampling:   7% (500/7000)
Sampling:   9% (600/7000)
Sampling:  10% (700/7000)
Sampling:  54% (3800/7000)
Sampling: 100% (7000/7000)
Sampling: 100% (7000/7000), done.
Messages received during sampling:
  Gradient evaluation took 0.000254 seconds
  1000 transitions using 10 leapfrog steps per transition would take 2.54 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.0003 seconds
  1000 transitions using 10 leapfrog steps per transition would take 3 seconds.
  Adjust your expectations accordingly!


simulated data shape: (42, 14)
Building...



Building: found in cache, done.Messages from stanc:
Sampling:   0%
Sampling: 100%, done.
Messages received during sampling:
  Gradient evaluation took 0.000254 seconds
  1000 transitions using 10 leapfrog steps per transition would take 2.54 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.0003 seconds
  1000 transitions using 10 leapfrog steps per transition would take 3 seconds.
  Adjust your expectations accordingly!


simulated data shape: (84, 14)
Building...


...


Building: found in cache, done.Messages from stanc:
Sampling:   0%
Sampling:   0% (1/7000)
Sampling:   0% (2/7000)
Sampling:   1% (101/7000)
Sampling:   3% (200/7000)
Sampling:   4% (300/7000)
Sampling:   6% (400/7000)
Sampling:   7% (500/7000)
Sampling:   9% (600/7000)
Sampling:  10% (700/7000)
Sampling:  11% (800/7000)
Sampling:  13% (901/7000)
Sampling:  14% (1002/7000)
Sampling:  16% (1101/7000)
Sampling:  17% (1200/7000)
Sampling:  19% (1300/7000)
Sampling:  20% (1400/7000)
Sampling:  21% (1500/7000)
Sampling:  23% (1600/7000)
Sampling:  24% (1700/7000)
Sampling:  26% (1800/7000)
Sampling:  27% (1900/7000)
Sampling:  29% (2000/7000)
Sampling:  30% (2100/7000)
Sampling:  31% (2200/7000)
Sampling:  33% (2300/7000)
Sampling:  34% (2400/7000)
Sampling:  36% (2500/7000)
Sampling:  37% (2600/7000)
Sampling:  39% (2700/7000)
Sampling:  40% (2800/7000)
Sampling:  41% (2900/7000)
Sampling:  43% (3000/7000)
Sampling:  44% (3100/7000)
Sampling:  46% (3200/7000)
Sampling:  47% (3300/7000)
Sa

simulated data shape: (840, 14)
Building...



Building: found in cache, done.Messages from stanc:
Sampling:   0%
Sampling:   0% (1/7000)
Sampling:   0% (2/7000)
Sampling:   1% (101/7000)
Sampling:   3% (200/7000)
Sampling:   4% (300/7000)
Sampling:   6% (400/7000)
Sampling:   7% (500/7000)
Sampling:   9% (600/7000)
Sampling:  10% (700/7000)
Sampling:  11% (800/7000)
Sampling:  13% (901/7000)
Sampling:  14% (1001/7000)
Sampling:  14% (1002/7000)
Sampling:  16% (1101/7000)
Sampling:  17% (1200/7000)
Sampling:  19% (1300/7000)
Sampling:  20% (1400/7000)
Sampling:  21% (1500/7000)
Sampling:  23% (1600/7000)
Sampling:  24% (1700/7000)
Sampling:  26% (1800/7000)
Sampling:  27% (1900/7000)
Sampling:  29% (2000/7000)
Sampling:  30% (2100/7000)
Sampling:  31% (2200/7000)
Sampling:  33% (2300/7000)
Sampling:  34% (2400/7000)
Sampling:  36% (2500/7000)
Sampling:  37% (2600/7000)
Sampling:  39% (2700/7000)
Sampling:  40% (2800/7000)
Sampling:  41% (2900/7000)
Sampling:  43% (3000/7000)
Sampling:  44% (3100/7000)
Sampling:  46% (3200/7000)
Sa

simulated data shape: (2100, 14)
Building...



Building: found in cache, done.Messages from stanc:
Sampling:   0%
Sampling:   0% (1/7000)
Sampling:   0% (2/7000)
Sampling:   1% (101/7000)
Sampling:   3% (200/7000)
Sampling:   4% (300/7000)
Sampling:   6% (400/7000)
Sampling:   7% (500/7000)
Sampling:   9% (600/7000)
Sampling:  10% (700/7000)
Sampling:  11% (800/7000)
Sampling:  13% (901/7000)
Sampling:  14% (1001/7000)
Sampling:  14% (1002/7000)
Sampling:  16% (1101/7000)
Sampling:  17% (1201/7000)
Sampling:  19% (1300/7000)
Sampling:  20% (1400/7000)
Sampling:  21% (1500/7000)
Sampling:  23% (1600/7000)
Sampling:  24% (1700/7000)
Sampling:  26% (1800/7000)
Sampling:  27% (1900/7000)
Sampling:  29% (2000/7000)
Sampling:  30% (2100/7000)
Sampling:  31% (2200/7000)
Sampling:  33% (2300/7000)
Sampling:  34% (2400/7000)
Sampling:  36% (2500/7000)
Sampling:  37% (2600/7000)
Sampling:  39% (2700/7000)
Sampling:  40% (2800/7000)
Sampling:  41% (2900/7000)
Sampling:  43% (3000/7000)
Sampling:  44% (3100/7000)
Sampling:  46% (3200/7000)
Sa

## Export Data

In [ ]:
path = "time_measured_pystan_data%s_model%s.csv" % (DATA, MODEL)

header = ['num_cases,start_time,end_time,elapsed_time']

with open(path, mode='w') as f:
    f.write('\n'.join(header + lines))

In [ ]:
from google.colab import files

files.download(path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>